##### Copyright 2023 Google LLC.

# Gemini API: Document Q&A with ChromaDB

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/chromadb/Vectordb_with_chroma.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/>

## Overview

Here we use the Gemini API to create a vector database and retrieve answers to questions from the database. We use [ChromaDB](https://docs.trychroma.com/), an open-source Python tool that creates embedding databases. ChromaDB allows you to:

* Store embeddings as well as their metadata
* Embed documents and queries
* Search through the database of embeddings

We embeddings to retrieve an answer from a database of vectors created with ChromaDB.

## Setup

First, download and install ChromaDB and the Gemini API Python library.

In [1]:
%pip install -U -q "google-generativeai>=0.7.2"
%pip install -q chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.0 MB/s eta 0:00:0

Then import the modules you'll use in this tutorial.

In [2]:
import textwrap
import chromadb
import numpy as np
import pandas as pd

import google.generativeai as genai

from IPython.display import Markdown
from chromadb import Documents, EmbeddingFunction, Embeddings

## Configure your API key

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.


In [3]:
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=api_key)

Key Point: Next, you will choose a model. Any embedding model will work for this tutorial, but for real applications it's important to choose a specific model and stick with it. The outputs of different models are not compatible with each other.

**Note**: At this time, the Gemini API is [only available in certain regions](https://ai.google.dev/available_regions).

In [4]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


### Data

Here is a small set of documents you will use to create an embedding database:

In [12]:
import json

def load_json_to_object(filepath):
  try:
    with open(filepath, 'r') as f:
      data = json.load(f)
      return data
  except FileNotFoundError:
    print(f"Error: File not found at {filepath}")
    return None
  except json.JSONDecodeError:
    print(f"Error: Invalid JSON format in {filepath}")
    return None

In [13]:
# load json file to str

import json

def load_json_to_str(filepath):
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
            return json.dumps(data) # Convert JSON object to a string
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {filepath}")
        return None


In [15]:
file_path = '/content/all_recipes.json'  # Replace with the actual path to your JSON file
all_recipes_data = load_json_to_object(file_path)

if all_recipes_data:
  print("JSON file loaded successfully into a Python object.")
  print(type(all_recipes_data))
else:
  print("Failed to load JSON file.")

JSON file loaded successfully into a Python object.
<class 'list'>


### Fonctionnement des **`Splitter`**

- La différence entre **`RecursiveCharacterTextSplitter`** et **`CharacterTextSplitter`** repose principalement sur la manière dont chacun gère la découpe des documents en fragments, notamment en ce qui concerne la structure des textes et la granularité du découpage.

- **`NotionDirectoryLoader`** est utilisé pour charger automatiquement les fichiers exportés depuis Notion, les rendant exploitables pour un traitement ultérieur, comme la recherche d'informations ou la génération de réponses.
- **`MarkdownHeaderTextSplitter`** découpe les fichiers Markdown en fonction des en-têtes structurés du texte, permettant un découpage logique des sections de contenu.

- Le **`TokenTextSplitter`** est utile lorsque vous souhaitez découper un texte en morceaux qui ne dépassent pas un certain nombre de tokens, plutôt qu’un certain nombre de caractères. Cela est important car la plupart des modèles de langage comme GPT ont une **limite de tokens** qu'ils peuvent traiter en une seule requête (par exemple, 4096 tokens pour GPT-3).

Ici nous utilisons RecursiveJsonSplitter comme notre data est en json

- This json splitter splits json data while allowing control over chunk sizes. It traverses json data depth first and builds smaller json chunks. It attempts to keep nested json objects whole but will split them if needed to keep chunks between a min_chunk_size and the max_chunk_size.
 -  How the text is split: json value.
 - How the chunk size is measured: by number of characters.

In [16]:
from langchain_text_splitters import RecursiveJsonSplitter
from langchain.text_splitter import TokenTextSplitter

json_splitter = RecursiveJsonSplitter(max_chunk_size=2048)

In [17]:
# print((all_recipes_data))

In [18]:
# Recursively split json data - If you need to access/manipulate the smaller json chunks
json_chunks = json_splitter.split_text(json_data=all_recipes_data, convert_lists=True)

In [19]:
for chunk in json_chunks[:3]:
    print(chunk)

{"0": {"author": "nadol", "canonical_url": "https://www.marmiton.org/recettes/recette_salade-de-courgettes-a-la-feta_18274.aspx", "category": null, "cook_time": 2, "cuisine": "Entr\u00e9e", "description": "courgette, feta, oignon, tomate, olives noires, basilic, poivre, sel", "host": "marmiton.org", "image": "https://assets.afcdn.com/recipe/20190704/94679_w1024h576c1cx2808cy1872cxt0cyt0cxb5616cyb3744.jpg", "ingredient_groups": {"0": {"ingredients": {"0": "2 courgettes", "1": "200 g de feta (sans huile)", "2": "2 oignons", "3": "4 tomates", "4": "olives noires", "5": "basilic", "6": "poivre", "7": "sel"}, "purpose": null}}, "ingredients": {"0": "2 courgettes", "1": "200 g de feta (sans huile)", "2": "2 oignons", "3": "4 tomates", "4": "olives noires", "5": "basilic", "6": "poivre", "7": "sel"}, "instructions": "Couper les courgettes en fines rondelles, et les faire d\u00e9gorger avec du gros sel dans une passoire environ 30 min (jusqu'\u00e0 ce qu'elles soient tendres).\nPendant ce temp

# 2nd way of splitting

In [20]:

!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.5 MB/s eta 0:00:00


In [21]:
# 2nd type of chunk avec token text splitter
# Specify max_chunk_size to constrain chunk sizes, limit of Gemini embedding is 2048 tokens
from langchain.text_splitter import TokenTextSplitter
import tiktoken

file_path = '/content/all_recipes.json'  # Replace with the actual path to your JSON file
all_recipes_str = load_json_to_str(file_path)

# Définir la taille et le chevauchement des tokens
chunk_size = 2000
chunk_overlap = 300

# Créer un splitter basé sur les tokens
token_splitter = TokenTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

# Utiliser le splitter sur le json format str
chunks_str = token_splitter.split_text(all_recipes_str)

print(type(chunks_str[0]))
print(chunks_str)

<class 'str'>
['[{"author": "nadol", "canonical_url": "https://www.marmiton.org/recettes/recette_salade-de-courgettes-a-la-feta_18274.aspx", "category": null, "cook_time": 2, "cuisine": "Entr\\u00e9e", "description": "courgette, feta, oignon, tomate, olives noires, basilic, poivre, sel", "host": "marmiton.org", "image": "https://assets.afcdn.com/recipe/20190704/94679_w1024h576c1cx2808cy1872cxt0cyt0cxb5616cyb3744.jpg", "ingredient_groups": [{"ingredients": ["2 courgettes", "200 g de feta (sans huile)", "2 oignons", "4 tomates", "olives noires", "basilic", "poivre", "sel"], "purpose": null}], "ingredients": ["2 courgettes", "200 g de feta (sans huile)", "2 oignons", "4 tomates", "olives noires", "basilic", "poivre", "sel"], "instructions": "Couper les courgettes en fines rondelles, et les faire d\\u00e9gorger avec du gros sel dans une passoire environ 30 min (jusqu\'\\u00e0 ce qu\'elles soient tendres).\\nPendant ce temps, couper la feta en petits cubes, couper les tomates en petits quar

## Creating the embedding database with ChromaDB

You will create a [custom function](https://docs.trychroma.com/embeddings#custom-embedding-functions){:.external} for performing embedding using the Gemini API. By inputting a set of documents into this custom function, you will receive vectors, or embeddings of the documents.


### API changes to Embeddings with model embedding-004

These new parameters apply only to the newest embeddings models.The task types are:

Task Type | Description
---       | ---
RETRIEVAL_QUERY	| Specifies the given text is a query in a search/retrieval setting.
RETRIEVAL_DOCUMENT | Specifies the given text is a document in a search/retrieval setting.
SEMANTIC_SIMILARITY	| Specifies the given text will be used for Semantic Textual Similarity (STS).
CLASSIFICATION	| Specifies that the embeddings will be used for classification.
CLUSTERING	| Specifies that the embeddings will be used for clustering.

In [22]:
class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    model = 'models/text-embedding-004'
    return genai.embed_content(model=model,
                                content=input,
                                task_type="semantic_similarity"
                                )["embedding"]

Now you will create the vector database. In the `create_chroma_db` function, you will instantiate a [Chroma client](https://docs.trychroma.com/getting-started){:.external}. From there, you will create a collection, which is where you store your embeddings, documents, and any metadata. Note that the embedding function from above is passed as an argument to the `create_collection`.

Next, you use the `add` method to add the documents to the collection.

In [23]:
import time
import random

def create_chroma_db(documents, name, dir):
  # chroma_client = chromadb.Client()
  persist_directory = dir
  chroma_client = chromadb.PersistentClient(path=persist_directory)
  db = chroma_client.get_or_create_collection(name=name,
                                       embedding_function=GeminiEmbeddingFunction())

  for i, d in enumerate(documents):
    time.sleep(0.5)
    db.add(
      documents=d,
      ids=str(i)
    )
  return db

In [ ]:
# chroma_client = chromadb.Client()
# chroma_client.delete_collection(name="marmiton_recipe")


In [24]:
persist_directory = './chroma2'

In [ ]:
# !rm -rf ./docs # remove old database files if any

In [25]:
# Set up the DB
db = create_chroma_db(chunks_str, "marmiton", persist_directory)

In [28]:
!zip -r chroma2.zip ./chroma2/

updating: chroma2/ (stored 0%)
updating: chroma2/chroma.sqlite3 (deflated 61%)
updating: chroma2/82fa7eb9-4e75-4a82-98f2-e0b2478c19a9/ (stored 0%)
updating: chroma2/82fa7eb9-4e75-4a82-98f2-e0b2478c19a9/length.bin (deflated 99%)
updating: chroma2/82fa7eb9-4e75-4a82-98f2-e0b2478c19a9/data_level0.bin (deflated 100%)
updating: chroma2/82fa7eb9-4e75-4a82-98f2-e0b2478c19a9/header.bin (deflated 61%)
updating: chroma2/82fa7eb9-4e75-4a82-98f2-e0b2478c19a9/link_lists.bin (stored 0%)


Confirm that the data was inserted by looking at the database:

In [29]:
pd.DataFrame(db.get()['documents'])

,0
0,"[{""author"": ""nadol"", ""canonical_url"": ""https:/..."
1,la sauce : verser le jus de citron dans un bo...
2,"parmesan)"", ""citron jaune"", ""3 cuill\u00e8res..."
3,"cuill\u00e8res \u00e0 soupe d'huile de noix"",..."
4,", tomate, poivre, sel, huile, citron"", ""host"":..."
...,...
457,", deux feuilles de basilic et un branchette de..."
458,u00fbt. Mettre au four 8 \u00e0 10 minutes \u0...
459,"pas trop....."", ""Laisser caram\u00e9liser l\u..."
460,"9g\u00e9e, aneth, ciboulette, jus de citron, h..."


## Getting the relevant document

`db` is a Chroma collection object. You can call `query` on it to perform a nearest neighbors search to find similar embeddings or documents.


In [30]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [31]:
def get_relevant_passage(query, db):
  passage = db.query(query_texts=[query], n_results=1)['documents'][0][0]
  return passage

In [32]:
# Perform embedding search
passage = get_relevant_passage("salade de fruit", db)
Markdown(passage)

 arroser de la vinaigrette \u00e0 la moutarde. R\u00e9server mais pas au frigo.", "Pr\u00e9lever le zeste du citron et de l\u2019orange \u00e0 l\u2019\u00e9conome. Les couper en une fine julienne puis en cubes minuscules. Les faire blanchir s\u00e9par\u00e9ment dans deux casseroles d\u2019eau bouillante pendant quelques minutes. Les \u00e9goutter et r\u00e9server.", "Couper les saumons en tout petits cubes. Hacher l\u2019aneth, les cornichons, les \u00e9chalotes, les c\u00e2pres.", "M\u00e9langer dans un saladier les saumons, le hachis, les zestes.", "Arroser avec l\u2019huile d\u2019olive et le jus du citron. Ajouter le piment, du sel et du poivre. R\u00e9server au frais.", "Au moment de servir, presser l\u2019orange. Filtrer et r\u00e9server le jus.", "Dresser \u00e0 l\u2019assiette la salade de lentilles. Ajouter dessus le tartare de saumon. Arroser de jus d\u2019orange. D\u00e9corer de brins d\u2019aneth."], "keywords": ["Tartare de saumon et sa salade de lentilles (17\u00e8me rencontre)", "tartare", "lentilles vertes", "saumon frais", "saumon fum\u00e9", "\u00e9chalote", "cornichons", "c\u00e2pres", "citron", "orange", "huile d'olive", "piment", "oignon", "clou de girofle", "thym", "laurier", "huile de noix", "vinaigre de x\u00e9r\u00e8s", "moutarde", "sel", "poivre blanc", "aneth", "tr\u00e8s facile", "moyen"], "language": "fr", "nutrients": {}, "prep_time": 40, "ratings": 4.9, "ratings_count": 82, "site_name": "Marmiton", "title": "Tartare de saumon et sa salade de lentilles (17\u00e8me rencontre)", "total_time": 60, "yields": "6 servings"}, {"author": "alya_148574", "canonical_url": "https://www.marmiton.org/recettes/recette_tartare-de-crevettes-mangue-et-avocats_195003.aspx", "category": "crevettes", "cook_time": null, "cuisine": "Entr\u00e9e", "description": "crevette rose, avocat, mangue, citron vert, oignon nouveau, huile d'olive, poivre, aneth, tabasco, fleur de sel", "host": "marmiton.org", "image": "https://assets.afcdn.com/recipe/20130121/35948_w1024h576c1cx813cy1000.jpg", "ingredient_groups": [{"ingredients": ["200 g de crevette rose cuites (plus 1 par personne pour la d\u00e9coration)", "2 avocats", "1 mangues", "1 citrons verts", "1 oignons nouveaux", "2 cuill\u00e8res d'huile d'olive", "1 demis de poivre", "4 brins d'aneth (plus quelques uns pour la d\u00e9coration)", "6 gouttes de tabasco", "fleur de sel de Gu\u00e9rande"], "purpose": null}], "ingredients": ["200 g de crevette rose cuites (plus 1 par personne pour la d\u00e9coration)", "2 avocats", "1 mangues", "1 citrons verts", "1 oignons nouveaux", "2 cuill\u00e8res d'huile d'olive", "1 demis de poivre", "4 brins d'aneth (plus quelques uns pour la d\u00e9coration)", "6 gouttes de tabasco", "fleur de sel de Gu\u00e9rande"], "instructions": "Ouvrez les avocats, retirez le noyau, pr\u00e9levez la chair, arrosez-la de jus de citron vert.\n\u00c9pluchez la mangue. Coupez-la tout en petits d\u00e9s.\nD\u00e9cortiquez les crevettes. Coupez-les \u00e9galement en petits morceaux.\nM\u00e9langez-les avec les d\u00e9s des fruits tr\u00e8s d\u00e9licatement, l'oignon hach\u00e9, le jus de citron vert restant, l'huile d'olive, les gouttes de tabasco ,le sel de Gu\u00e9rande (selon convenance) et le poivre \u00e9cras\u00e9 et l'aneth coup\u00e9e.\nDisposez cette pr\u00e9paration dans vos coupelles, assiettes de pr\u00e9sentation... parsemez de poivre, fleur de sel de Gu\u00e9rande et aneth !", "instructions_list": ["Ouvrez les avocats, retirez le noyau, pr\u00e9levez la chair, arrosez-la de jus de citron vert.", "\u00c9pluchez la mangue. Coupez-la tout en petits d\u00e9s.", "D\u00e9cortiquez les crevettes. Coupez-les \u00e9galement en petits morceaux.", "M\u00e9langez-les avec les d\u00e9s des fruits tr\u00e8s d\u00e9licatement, l'oignon hach\u00e9, le jus de citron vert restant, l'huile d'olive, les gouttes de tabasco ,le sel de Gu\u00e9rande (selon convenance) et le poivre \u00e9cras\u00e9 et l'aneth coup\u00e9e.", "Disposez cette pr\u00e9paration dans vos coupelles, assiettes de pr\u00e9sentation... parsemez de poivre, fleur de sel de Gu\u00e9rande et aneth !"], "keywords": ["Tartare de crevettes mangue avocats", "crevettes", "crevette rose", "avocat", "mangue", "citron vert", "oignon nouveau", "huile d'olive", "poivre", "aneth", "tabasco", "fleur de sel", "moyenne", "moyen", "rapide"], "language": "fr", "nutrients": {}, "prep_time": 40, "ratings": 4.9, "ratings_count": 84, "site_name": "Marmiton", "title": "Tartare de crevettes mangue avocats", "total_time": 40, "yields": "4 servings"}, {"author": "el_Fakir", "canonical_url": "https://www.marmiton.org/recettes/recette_steak-tartare_18121.aspx", "category": "pi\u00e8ce de boeuf", "cook_time": null, "cuisine": "Plat principal", "description": "boeuf, jaune d'oe

Now that you have found the relevant passage in your set of documents, you can use it make a prompt to pass into the Gemini API.

In [33]:
def cook_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = ("""Parlons cuisine ! Tu es mon assistant culinaire amical et serviable. \
  As-tu des idées pour une recette simple et délicieuse ? \
  Base toi sur les recettes que tu connais et donne moi autant de details que possible\
  pour t'aider à trouver la meilleure suggestion.\n\n \
  QUESTION: '{query}' \n\n
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt
# N'hésite pas à me poser des questions sur mes préférences ou sur les ingrédients que j'ai sous la main \
  # # Je pense préparer '{type_de_repas}' ce soir, peut-être quelque chose avec '{ingrédient_principal}'. \

Pass a query to the prompt:

In [36]:
question = "Quelle type de salade de fruit je peux faire ? donne moi 3 exemples"
prompt = cook_prompt(question, passage)
Markdown(prompt)

Parlons cuisine ! Tu es mon assistant culinaire amical et serviable.   As-tu des idées pour une recette simple et délicieuse ?   Base toi sur les recettes que tu connais et donne moi autant de details que possible  pour t'aider à trouver la meilleure suggestion.

   QUESTION: 'Quelle type de salade de fruit je peux faire ? donne moi 3 exemples' 


  PASSAGE: ' arroser de la vinaigrette \u00e0 la moutarde. R\u00e9server mais pas au frigo., Pr\u00e9lever le zeste du citron et de l\u2019orange \u00e0 l\u2019\u00e9conome. Les couper en une fine julienne puis en cubes minuscules. Les faire blanchir s\u00e9par\u00e9ment dans deux casseroles d\u2019eau bouillante pendant quelques minutes. Les \u00e9goutter et r\u00e9server., Couper les saumons en tout petits cubes. Hacher l\u2019aneth, les cornichons, les \u00e9chalotes, les c\u00e2pres., M\u00e9langer dans un saladier les saumons, le hachis, les zestes., Arroser avec l\u2019huile d\u2019olive et le jus du citron. Ajouter le piment, du sel et du poivre. R\u00e9server au frais., Au moment de servir, presser l\u2019orange. Filtrer et r\u00e9server le jus., Dresser \u00e0 l\u2019assiette la salade de lentilles. Ajouter dessus le tartare de saumon. Arroser de jus d\u2019orange. D\u00e9corer de brins d\u2019aneth.], keywords: [Tartare de saumon et sa salade de lentilles (17\u00e8me rencontre), tartare, lentilles vertes, saumon frais, saumon fum\u00e9, \u00e9chalote, cornichons, c\u00e2pres, citron, orange, huile dolive, piment, oignon, clou de girofle, thym, laurier, huile de noix, vinaigre de x\u00e9r\u00e8s, moutarde, sel, poivre blanc, aneth, tr\u00e8s facile, moyen], language: fr, nutrients: {}, prep_time: 40, ratings: 4.9, ratings_count: 82, site_name: Marmiton, title: Tartare de saumon et sa salade de lentilles (17\u00e8me rencontre), total_time: 60, yields: 6 servings}, {author: alya_148574, canonical_url: https://www.marmiton.org/recettes/recette_tartare-de-crevettes-mangue-et-avocats_195003.aspx, category: crevettes, cook_time: null, cuisine: Entr\u00e9e, description: crevette rose, avocat, mangue, citron vert, oignon nouveau, huile dolive, poivre, aneth, tabasco, fleur de sel, host: marmiton.org, image: https://assets.afcdn.com/recipe/20130121/35948_w1024h576c1cx813cy1000.jpg, ingredient_groups: [{ingredients: [200 g de crevette rose cuites (plus 1 par personne pour la d\u00e9coration), 2 avocats, 1 mangues, 1 citrons verts, 1 oignons nouveaux, 2 cuill\u00e8res dhuile dolive, 1 demis de poivre, 4 brins daneth (plus quelques uns pour la d\u00e9coration), 6 gouttes de tabasco, fleur de sel de Gu\u00e9rande], purpose: null}], ingredients: [200 g de crevette rose cuites (plus 1 par personne pour la d\u00e9coration), 2 avocats, 1 mangues, 1 citrons verts, 1 oignons nouveaux, 2 cuill\u00e8res dhuile dolive, 1 demis de poivre, 4 brins daneth (plus quelques uns pour la d\u00e9coration), 6 gouttes de tabasco, fleur de sel de Gu\u00e9rande], instructions: Ouvrez les avocats, retirez le noyau, pr\u00e9levez la chair, arrosez-la de jus de citron vert.\n\u00c9pluchez la mangue. Coupez-la tout en petits d\u00e9s.\nD\u00e9cortiquez les crevettes. Coupez-les \u00e9galement en petits morceaux.\nM\u00e9langez-les avec les d\u00e9s des fruits tr\u00e8s d\u00e9licatement, loignon hach\u00e9, le jus de citron vert restant, lhuile dolive, les gouttes de tabasco ,le sel de Gu\u00e9rande (selon convenance) et le poivre \u00e9cras\u00e9 et laneth coup\u00e9e.\nDisposez cette pr\u00e9paration dans vos coupelles, assiettes de pr\u00e9sentation... parsemez de poivre, fleur de sel de Gu\u00e9rande et aneth !, instructions_list: [Ouvrez les avocats, retirez le noyau, pr\u00e9levez la chair, arrosez-la de jus de citron vert., \u00c9pluchez la mangue. Coupez-la tout en petits d\u00e9s., D\u00e9cortiquez les crevettes. Coupez-les \u00e9galement en petits morceaux., M\u00e9langez-les avec les d\u00e9s des fruits tr\u00e8s d\u00e9licatement, loignon hach\u00e9, le jus de citron vert restant, lhuile dolive, les gouttes de tabasco ,le sel de Gu\u00e9rande (selon convenance) et le poivre \u00e9cras\u00e9 et laneth coup\u00e9e., Disposez cette pr\u00e9paration dans vos coupelles, assiettes de pr\u00e9sentation... parsemez de poivre, fleur de sel de Gu\u00e9rande et aneth !], keywords: [Tartare de crevettes mangue avocats, crevettes, crevette rose, avocat, mangue, citron vert, oignon nouveau, huile dolive, poivre, aneth, tabasco, fleur de sel, moyenne, moyen, rapide], language: fr, nutrients: {}, prep_time: 40, ratings: 4.9, ratings_count: 84, site_name: Marmiton, title: Tartare de crevettes mangue avocats, total_time: 40, yields: 4 servings}, {author: el_Fakir, canonical_url: https://www.marmiton.org/recettes/recette_steak-tartare_18121.aspx, category: pi\u00e8ce de boeuf, cook_time: null, cuisine: Plat principal, description: boeuf, jaune doe'

    ANSWER:
  

Now use the `generate_content` method to to generate a response from the model.

In [37]:
model = genai.GenerativeModel('gemini-2.0-flash')
answer = model.generate_content(prompt)
Markdown(answer.text)

Bien sûr ! Voici 3 idées de salades de fruits simples et délicieuses, en m'inspirant des éléments que tu m'as fournis, et en y ajoutant ma touche personnelle :

1.  **Salade de Fruits Exotique à la Vinaigrette Citron-Aneth (Inspirée du tartare de saumon):**
    *   **Fruits:** Mangue (en dés), Ananas (en dés), Kiwi (en tranches), Fruit de la passion (pulpe)
    *   **Vinaigrette:** Jus de citron vert, huile d'olive, miel (ou sirop d'érable), aneth hachée finement.
    *   **Préparation:** Mélange délicatement les fruits. Prépare la vinaigrette en fouettant ensemble les ingrédients. Verse la vinaigrette sur les fruits juste avant de servir.
    *   **Pourquoi c'est bon:** L'acidité du citron vert et la fraîcheur de l'aneth se marient parfaitement avec la douceur des fruits exotiques.
2.  **Salade de Fruits Rouges Rafraîchissante à la Menthe:**
    *   **Fruits:** Fraises (coupées en deux ou en quatre), Framboises, Myrtilles, Mûres
    *   **Optionnel:** Quelques cerises dénoyautées et coupées en deux
    *   **Assaisonnement:** Jus de citron, feuilles de menthe fraîche ciselées, une pincée de sucre glace (facultatif)
    *   **Préparation:** Lave et coupe les fruits si nécessaire. Mélange-les délicatement. Arrose de jus de citron et saupoudre de menthe fraîche. Ajoute une pincée de sucre glace si tu souhaites une salade plus sucrée.
    *   **Pourquoi c'est bon:** Les fruits rouges sont naturellement sucrés et acidulés, et la menthe apporte une touche de fraîcheur très agréable.
3.  **Salade de Fruits d'Automne Épicée à l'Orange:**
    *   **Fruits:** Pommes (en dés), Poires (en dés), Raisins (coupés en deux), Oranges (en segments)
    *   **Épices:** Une pincée de cannelle, de gingembre moulu, et une touche de clou de girofle (très léger pour ne pas dominer)
    *   **Préparation:** Coupe les fruits en morceaux de taille similaire. Mélange-les délicatement. Dans un petit bol, mélange le jus d'orange avec les épices. Verse ce mélange sur les fruits et mélange doucement.
    *   **Pourquoi c'est bon:** Les fruits d'automne ont une saveur plus riche et les épices chaudes mettent en valeur leur douceur naturelle. L'orange apporte une touche d'acidité et d'éclat.

J'espère que ces suggestions te plaisent ! N'hésite pas à me poser d'autres questions si tu souhaites affiner une recette ou en explorer d'autres. 😊


## Next steps

To learn more about how you can use the embeddings, check out the [examples](https://ai.google.dev/examples?keywords=embed) available. To learn how to use other services in the Gemini API, visit the [Python quickstart](https://ai.google.dev/gemini-api/docs/get-started/python).